In [85]:
import jutil
from sklearn.cluster import KMeans
import numpy as np
from collections import Counter

In [86]:
sql = "select t.Course_code as course_code from train_plan_course t,train_plan_credit c where t.Course_big_type_id=c.sid \
and t.Grade='2014' and c.type_name like '%专业%'  and t.Speciality_code='0408'"
code = [d['course_code'] for d in jutil.load_data(sql)]

In [87]:
len(set(code))

53

In [88]:
sql = "select * from view_stu_course_mark where speciality_code='0408' and grade='2014'"
df = jutil.load_pd_df(sql)
df.head()

,birthday,city,class_code,class_name,college_code,college_name,course_code,course_name,course_type,exam_type,...,province,sex,speciality_code,speciality_name,student_id,teach_class_id,teacher,term_id,term_name,term_order
0,710006400000,聊城市,0408201402,软件2014-02班,04,信息科学与技术学院,0471072,数据库原理实验,必,正考,...,山东,男,0408,软件工程,20122639,1,陶宏才,79,2016-2017第1学期,3
1,710006400000,聊城市,0408201402,软件2014-02班,04,信息科学与技术学院,0473082,数据库技术与应用,必,正考,...,山东,男,0408,软件工程,20122639,1,黄洪,80,2016-2017第2学期,4
2,710006400000,聊城市,0408201402,软件2014-02班,04,信息科学与技术学院,0474020,数据库技术与应用实验,必,正考,...,山东,男,0408,软件工程,20122639,1,黄洪,80,2016-2017第2学期,4
3,710006400000,聊城市,0408201402,软件2014-02班,04,信息科学与技术学院,0871112,职场英语,限,正考,...,山东,男,0408,软件工程,20122639,20,刘波,76,2015-2016第2学期,3
4,710006400000,聊城市,0408201402,软件2014-02班,04,信息科学与技术学院,1371032,大学物理BI,必,正考,...,山东,男,0408,软件工程,20122639,1,崔占涛,72,2014-2015第2学期,2


In [89]:
# 选出了与专业相关的课程
dd = df[df['course_code'].isin(code)]

## 按学生group,然后将其课程分数作为维度,学生数作为数据量

In [101]:
m = dd.drop_duplicates(subset=['student_id','course_code']).groupby(['student_id','name'],as_index=False)['pmark'].apply(list)
md = m.to_dict()
marks = list(m)

In [113]:
# 得到大多数同学都有的课程
cnt = [len(m) for m in marks]
most_len = max(set(cnt),key=cnt.count)
# most_len
cc = dd.drop_duplicates(subset=['student_id','course_code']).groupby(['student_id','name'],as_index=False)['course_name'].apply(list)
course_names = []
for k,v in cc.iteritems():
    if len(v)==most_len:
        course_names = v
        break
course_names

['面向对象程序设计实验',
 '计算机组成原理B',
 '数据库技术与应用',
 '计算机图形学实验',
 '操作系统实验',
 '计算机组成实验C',
 '软件项目管理',
 '面向对象程序设计',
 '程序语言综合课程设计',
 '数据库原理',
 '数据库原理实验',
 '微机与接口技术A',
 '人机交互技术',
 '人机交互技术实验',
 'Java程序设计',
 '计算方法C',
 '数据库技术与应用实验',
 '数字电子技术B',
 '高级语言程序设计',
 '微机与接口技术实验',
 '软件工程概论',
 '软件系统分析与设计',
 '数据结构实验',
 '计算机导论A',
 '算法设计与分析',
 '大学物理实验Ⅰ',
 'Java程序设计实验',
 '高级语言程序设计实验',
 '软件质量保证与测试',
 'J2EE体系结构及程序设计',
 '新技术讲座',
 '数据结构A',
 '计算机图形学',
 '高等数学Ⅰ',
 '高等数学Ⅱ',
 '数学建模B',
 '离散数学A',
 '软件工程/网络软件开发实习',
 '计算机网络工程实验',
 '数字电子技术实验B(电类)',
 '线性代数B',
 '大学物理实验Ⅱ',
 '软件设计实习']

In [115]:
gd = dd.drop_duplicates(subset=['student_id','course_code']).groupby(['student_id','name'],as_index=False)
d_arr = dict() # key存的是课程名,value是存放成绩的数组,这样竖着看就是一个学生的成绩列表了,也就是转置操作
for name ,g in gd:
    d_m = dict()
    for i,row in g.iterrows():
        d_m[row['course_name']] = row['pmark']
    for c in course_names:
        d_arr[c] = d_arr.get(c,[])
        d_arr[c].append(d_m.get(c,60))
courses = d_arr.keys()
mt = [arr for arr in d_arr.values()]
mt = np.transpose(mt)

In [116]:
mt

array([[73. , 90. , 91. , ..., 94. , 80.7, 87. ],
       [62. , 60. , 73. , ..., 48. ,  5.6, 60. ],
       [68. , 60. , 60. , ..., 70. , 75.7, 60. ],
       ...,
       [91. , 81. , 60. , ..., 74. , 83. , 62. ],
       [72. , 80. , 79. , ..., 75. , 76.2, 83. ],
       [78. , 70. , 82. , ..., 71. , 60. , 80. ]])

## 使用kmeans进行聚类

In [117]:
X = np.array(mt)

In [122]:
kmeans = KMeans(n_clusters=4).fit(X)

In [123]:
kmeans.labels_

array([3, 2, 2, 2, 1, 3, 3, 1, 2, 1, 1, 1, 1, 0, 3, 0, 1, 3, 0, 1, 3, 0,
       1, 3, 3, 1, 1, 1, 1, 0, 3, 0, 0, 3, 0, 3, 3, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 0, 3, 0, 3, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 3, 1, 0, 1,
       3, 3, 1, 0, 1, 1, 0, 1, 3, 3, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 3, 3, 1, 3, 0, 3, 0, 2, 1, 2, 0, 3, 3, 1, 0, 1, 0, 1, 3,
       0, 3, 3, 0, 2, 3, 0, 2, 0, 0], dtype=int32)

In [124]:
kmeans.cluster_centers_

array([[76.02222222, 67.92      , 83.6       , 84.75555556, 82.91111111,
        78.44444444, 81.31111111, 81.24444444, 83.2       , 76.37777778,
        86.37777778, 68.66666667, 75.84444444, 82.04444444, 80.91111111,
        65.17777778, 85.        , 87.35555556, 81.15555556, 75.31111111,
        69.75555556, 76.82222222, 88.17777778, 78.37777778, 68.46666667,
        80.82222222, 92.91111111, 75.66666667, 77.22222222, 80.31111111,
        78.77777778, 79.11111111, 73.04444444, 84.06666667, 75.77777778,
        81.22222222, 73.75555556, 82.24444444, 71.88888889, 85.62222222,
        79.28888889, 77.66222222, 81.42222222],
       [68.94736842, 60.79236842, 81.28947368, 83.47368421, 82.07894737,
        70.15789474, 77.10526316, 71.15789474, 79.63157895, 68.5       ,
        85.81578947, 61.23684211, 75.        , 80.18421053, 76.86842105,
        56.39473684, 82.34210526, 75.47368421, 68.89473684, 68.71052632,
        64.57894737, 72.60526316, 86.10526316, 75.02631579, 59.10526316,
   

## 进行信息整合
将学生和分类对应起来

In [125]:
cls_index = list(kmeans.labels_)
keys = list(md)
stu = {}
for i in range(len(cls_index)):
    stu[cls_index[i]] = stu.get(cls_index[i],[])
    stu[cls_index[i]].append(keys[i][1])
stu

{0: ['杨筱都',
  '朱烨',
  '范曾',
  '林于琨',
  '王天豪',
  '魏以彬',
  '李彦豫',
  '刘萱',
  '李强',
  '任挽澜',
  '孟祥印',
  '韩小川',
  '黄腾',
  '苏楠',
  '韩庆华',
  '孙文茹',
  '吕文琪',
  '焦俊嘉',
  '马君宜',
  '李蒙',
  '黄铭',
  '杨海朋',
  '李天峒',
  '喻盼盼',
  '周锦',
  '刘维仪',
  '杨敏松',
  '肖平',
  '杨鹏',
  '黄钲林',
  '戴一浩',
  '陈旭钢',
  '陈濛',
  '季敏东',
  '王奕杰',
  '吕凯莉',
  '李谢清',
  '朱秀林',
  '唐莹',
  '黄鑫',
  '张翔',
  '朱浩楠',
  '宋正聪',
  '王楚',
  '吴君正'],
 1: ['林琳',
  '张元谨',
  '屈春健',
  '赵凡',
  '万润聪',
  '李志鹏',
  '冶振华',
  '王雅坤',
  '黄强',
  '徐坤',
  '李红锐',
  '张家余',
  '李凯',
  '赵鹏宇',
  '罗鹏',
  '丛家声',
  '黄贵伟',
  '余彦乔',
  '陈宇航',
  '李健田',
  '周虎祥',
  '伍伶俐',
  '李晨',
  '陈双艳',
  '吴志刚',
  '杨雨栋',
  '刘中轩',
  '马亚',
  '刘亚飞',
  '高雨',
  '何军',
  '刘峰',
  '董凌飞',
  '游浩然',
  '吴宇航',
  '马昊',
  '单源超',
  '曾令东'],
 2: ['庄勇',
  '张逊桥',
  '阮林恒',
  '余思瀚                                            ',
  '熊义靖',
  '丁智弘',
  '努尔江·叶克奔',
  '凡远安'],
 3: ['杨永虎',
  '丁文琦',
  '纪圣荣',
  '左浩宇',
  '李林翰',
  '雷宁',
  '蒋妮妮',
  '张人之',
  '王棚',
  '黄鑫',
  '韩煜',
  '孟详淞',
  '潘晓波',
  '吴俊宇',
  '李俊晖',
  '王广兴',
  '陈钢霖'

In [84]:
center = kmeans.cluster_centers_
cls = {}
for i in range(3):
    cls[i] = dict()
    cls[i]['stu'] = stu[i]
    cls[i]['center'] = list(center[i])
cls

{0: {'center': [76.67999999999999,
   75.22,
   74.568,
   74.744,
   76.328,
   74.282,
   74.746,
   68.96400000000001,
   73.44,
   74.48,
   76.42,
   72.176,
   74.454,
   72.4122,
   75.56,
   74.028,
   74.424,
   75.85,
   75.4,
   72.708,
   71.85999999999999,
   71.5,
   72.084,
   71.94,
   74.26,
   76.01,
   74.14,
   76.94,
   73.26,
   72.66,
   74.502,
   74.248,
   74.52000000000001,
   75.6,
   75.094,
   74.574,
   74.00999999999999,
   75.98,
   74.54,
   74.632,
   75.676,
   72.348,
   71.50200000000001,
   60.298],
  'stu': ['林琳',
   '张元谨',
   '屈春健',
   '赵凡',
   '万润聪',
   '李志鹏',
   '杨筱都',
   '冶振华',
   '王雅坤',
   '林于琨',
   '黄强',
   '徐坤',
   '李红锐',
   '张家余',
   '李凯',
   '李彦豫',
   '任挽澜',
   '赵鹏宇',
   '韩庆华',
   '丛家声',
   '孙文茹',
   '黄贵伟',
   '余彦乔',
   '李蒙',
   '陈宇航',
   '李健田',
   '周虎祥',
   '伍伶俐',
   '李晨',
   '陈双艳',
   '吴志刚',
   '杨雨栋',
   '刘中轩',
   '马亚',
   '刘亚飞',
   '杨敏松',
   '高雨',
   '何军',
   '杨鹏',
   '刘峰',
   '董凌飞',
   '戴一浩',
   '陈旭钢',
   '游浩然',
   '吴宇航',
   '马昊',
  